<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Подготовка" data-toc-modified-id="Подготовка-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Подготовка</a></span></li><li><span><a href="#Обучение" data-toc-modified-id="Обучение-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Обучение</a></span></li><li><span><a href="#Выводы" data-toc-modified-id="Выводы-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Выводы</a></span></li><li><span><a href="#Чек-лист-проверки" data-toc-modified-id="Чек-лист-проверки-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Чек-лист проверки</a></span></li></ul></div>

# Проект для «Викишоп»

Интернет-магазин «Викишоп» запускает новый сервис. Теперь пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах. То есть клиенты предлагают свои правки и комментируют изменения других. Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию. 

Обучите модель классифицировать комментарии на позитивные и негативные. В вашем распоряжении набор данных с разметкой о токсичности правок.

Постройте модель со значением метрики качества *F1* не меньше 0.75. 

**Инструкция по выполнению проекта**

1. Загрузите и подготовьте данные.
2. Обучите разные модели. 
3. Сделайте выводы.

Для выполнения проекта применять *BERT* необязательно, но вы можете попробовать.

**Описание данных**

Данные находятся в файле `toxic_comments.csv`. Столбец *text* в нём содержит текст комментария, а *toxic* — целевой признак.

## Подготовка

In [1]:
# загрузим библиотеки
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import os
import re

import nltk 
from nltk.corpus import stopwords, wordnet
from nltk.stem import WordNetLemmatizer

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import (train_test_split, 
                                     TimeSeriesSplit, 
                                     GridSearchCV)
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import f1_score
from sklearn.pipeline import Pipeline

from tqdm.notebook import tqdm

import warnings
warnings.filterwarnings('ignore')

In [2]:
# загрузим данные
# первый столбец в данных - индекс
path1 = '/datasets/toxic_comments.csv'
path2 = '/Users/alex/Library/Mobile Documents/com~apple~CloudDocs/Education/Яндекс_Практикум/Проекты/19_Машинное_обучение_для_текстов/toxic_comments.csv'

if os.path.exists(path1):
    df = pd.read_csv(path1, index_col=0)
    print('Данные загружены с сервера Яндекс Практикума.')
elif os.path.exists(path2, index_col=0):
    df = pd.read_csv(path2)
    print('Данные загружены локально.')
else:
    print('Данные не загружены.')

Данные загружены с сервера Яндекс Практикума.


In [3]:
# посмотрим на данные
df.info()
display(df.sample(3))

<class 'pandas.core.frame.DataFrame'>
Int64Index: 159292 entries, 0 to 159450
Data columns (total 2 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   text    159292 non-null  object
 1   toxic   159292 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 3.6+ MB


,text,toxic
156759,Stroke your Sockpuppet\nPlease can I? PLEEEAAA...,0
95085,Scandinavia \nYou're welcome to take part in t...,0
68954,So what's the problem? Perhaps the affermation...,0


In [4]:
# проверим дисбаланс целевого признака
print(f"Класс 0: {df['toxic'].value_counts()[0] / len(df):.2%}")
print(f"Класс 1: {df['toxic'].value_counts()[1] / len(df):.2%}")

Класс 0: 89.84%
Класс 1: 10.16%


In [5]:
# предобработаем текст
# функция для создания тега частей речи
# функция для обработки и лемматизации текста

nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')
nltk.download('stopwords')

stop_words = set(stopwords.words('english'))

def get_wordnet_pos(word):
    tag = nltk.pos_tag([word])[0][1][0].upper()
    tag_dict = {"J": wordnet.ADJ,
                "N": wordnet.NOUN,
                "V": wordnet.VERB,
                "R": wordnet.ADV}
    return tag_dict.get(tag, wordnet.NOUN)
                        
def word_processing(some_text):
    lemmatizer = WordNetLemmatizer()
    sentence1 = re.sub(r'[^a-zA-Z]', ' ', some_text.lower())
    sentence2 = list(lemmatizer.lemmatize(w, get_wordnet_pos(w)) for w in nltk.word_tokenize(sentence1))
    sentence3 = ' '.join(sentence2)
    
    return ' '.join(sentence3.split())

tqdm.pandas()
df['text_lem'] = df['text'].progress_apply(word_processing)

[nltk_data] Downloading package punkt to /home/jovyan/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/jovyan/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to /home/jovyan/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /home/jovyan/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


  0%|          | 0/159292 [00:00<?, ?it/s]

In [6]:
# создадим обучающую и тестовую выборки: 8:2
# преобразуем текст в массив признаков с помощью TF-IDF
random_state = 42
features = df['text_lem']
target = df['toxic']

X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.2,
                                                    random_state=random_state, stratify=target)

print('Обучающая:', X_train.shape)
print('Тестовая:', X_test.shape)

Обучающая: (127433,)
Тестовая: (31859,)


* В таблице с предоставленными данными `159292` записей, пропусков нет;
* Дисбаланс целевого признака: чуть более `10%` токсичных комментариев;
* Язык комментариев: английский;
* В тексте содержатся лишние символы.    

Данные успешно загружены, очищены от лишних символов, приведены к одному формату и лемматизированы, выделены обучающая и тестовая выборки (`8:2`).

## Обучение

In [7]:
# создадим обучающую и тестовую выборки: 8:2
# преобразуем текст в массив признаков с помощью TF-IDF
# обучим три модели:  LogisticRegression, SGDClassifier, DecisionTreeClassifier
# метрика F1

tfidf_params = {'stop_words': stop_words}

params = [{'clf':[LogisticRegression(class_weight='balanced', random_state=random_state)],
           'clf__penalty':['l1', 'l2'],
           'clf__C':[0.01,1,10],
           'clf__solver': ['liblinear', 'sag', 'saga']
          },
          
          {'clf': [SGDClassifier(class_weight='balanced', random_state=random_state)],
           'clf__loss': ('hinge', 'log', 'modified_huber'),
           'clf__learning_rate': ('constant', 'optimal', 'invscaling', 'adaptive'),
           'clf__eta0': (.001, .05, .1)
          },
          
          {'clf': [DecisionTreeClassifier(class_weight='balanced', random_state=random_state)],
           'clf__criterion':['gini', 'entropy'],
           'clf__max_depth': np.arange(10, 50, 10).tolist(),
           'clf__min_samples_leaf': np.arange(1, 3, 1).tolist()
          }
         ]

best_scores = {}

models = {'LogisticRegression': LogisticRegression(class_weight='balanced', random_state=random_state),
          'SGDClassifier': SGDClassifier(class_weight='balanced', random_state=random_state),
          'DecisionTreeClassifier': DecisionTreeClassifier(class_weight='balanced', random_state=random_state)}

pipeline = Pipeline([('tf_idf', TfidfVectorizer(**tfidf_params)), ('clf', LogisticRegression())])

grid_search = GridSearchCV(pipeline, params, cv=2, scoring='f1', verbose=5)
grid_search.fit(X_train, y_train)
                         
best_model = grid_search.best_estimator_

print()

print(f"{best_model[1]}: Score = {grid_search.best_score_:.4f}")

Fitting 2 folds for each of 70 candidates, totalling 140 fits
[CV 1/2] END clf=LogisticRegression(class_weight='balanced', random_state=42), clf__C=0.01, clf__penalty=l1, clf__solver=liblinear; total time=   6.2s
[CV 2/2] END clf=LogisticRegression(class_weight='balanced', random_state=42), clf__C=0.01, clf__penalty=l1, clf__solver=liblinear; total time=   6.3s
[CV 1/2] END clf=LogisticRegression(class_weight='balanced', random_state=42), clf__C=0.01, clf__penalty=l1, clf__solver=sag; total time=   3.0s
[CV 2/2] END clf=LogisticRegression(class_weight='balanced', random_state=42), clf__C=0.01, clf__penalty=l1, clf__solver=sag; total time=   3.0s
[CV 1/2] END clf=LogisticRegression(class_weight='balanced', random_state=42), clf__C=0.01, clf__penalty=l1, clf__solver=saga; total time=   8.2s
[CV 2/2] END clf=LogisticRegression(class_weight='balanced', random_state=42), clf__C=0.01, clf__penalty=l1, clf__solver=saga; total time=   7.8s
[CV 1/2] END clf=LogisticRegression(class_weight='bala

Лучшая модель:    
`LogisticRegression: Score = 0.7559, Params = {'C': 10, 'penalty': 'l2', 'solver': 'liblinear'}`

In [13]:
# проверим модель на тестовой выборке
results = []
predictions_valid = best_model.predict(X_test)
f1 =  f1_score(y_test, predictions_valid)
results.append({'Модель': best_model[1].__class__.__name__,
                'F1': f1})
pd.DataFrame(results)

,Модель,F1
0,LogisticRegression,0.757901


## Выводы

* Загружен и проанализирован набор данных с разметкой о токсичности правок.
* Текстовые данные очищены от лишних символов, приведены к одному формату и лемматизированы, посчитана частота слов и выполнена кодировка в массив признаков.
* Из трёх обученных моделей выбрана наилучшая: `LogisticRegression`, с гиперпараметрами:
* * `C=10, penalty='l2', solver='liblinear', class_weight='balanced'` .
* На тестовых данных, значение метрики `F1 = 0.757901`, что удовлетворяет условиям поставленной задачи.
